In [1]:
#import funconnect
import funconnect

Connecting celiib@10.28.0.34:3306


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import datajoint as dj
from funconnect import connectomics as cncs, ta3, nda
from funconnect.connectomics import *
from funconnect import nda
import numpy as np
import pandas as pd
import pycircstat as circ
cncs.schema.spawn_missing_classes()

In [3]:
Syn = ta3.Synapse()
presyn_ids = Syn.fetch("presyn")
#presyn_ids.describe()

In [4]:
Syn.describe()

# Anatomically localized synapse between two Segments
-> ta3.Segmentation
synapse_id           : bigint                       # synapse index within the segmentation
---
-> ta3.Segment.proj(presyn="segment_id")
-> ta3.Segment.proj(postsyn="segment_id")
synapse_x            : bigint                       # (EM voxels)
synapse_y            : bigint                       # (EM voxels)
synapse_z            : bigint                       # (EM voxels)
syn_bbox_x1          : bigint                       # (EM voxels)  - bounding box
syn_bbox_y1          : bigint                       # (EM voxels)  - bounding box
syn_bbox_z1          : bigint                       # (EM voxels)  - bounding box
syn_bbox_x2          : bigint                       # (EM voxels)  - bounding box
syn_bbox_y2          : bigint                       # (EM voxels)  - bounding box
syn_bbox_z2          : bigint                       # (EM voxels)  - bounding box



'# Anatomically localized synapse between two Segments\n-> ta3.Segmentation\nsynapse_id           : bigint                       # synapse index within the segmentation\n---\n-> ta3.Segment.proj(presyn="segment_id")\n-> ta3.Segment.proj(postsyn="segment_id")\nsynapse_x            : bigint                       # (EM voxels)\nsynapse_y            : bigint                       # (EM voxels)\nsynapse_z            : bigint                       # (EM voxels)\nsyn_bbox_x1          : bigint                       # (EM voxels)  - bounding box\nsyn_bbox_y1          : bigint                       # (EM voxels)  - bounding box\nsyn_bbox_z1          : bigint                       # (EM voxels)  - bounding box\nsyn_bbox_x2          : bigint                       # (EM voxels)  - bounding box\nsyn_bbox_y2          : bigint                       # (EM voxels)  - bounding box\nsyn_bbox_z2          : bigint                       # (EM voxels)  - bounding box\n'

In [4]:
from collections import Counter
myCounter = Counter(presyn_ids)
myCounter

Counter({31: 4,
         71: 2,
         108: 3,
         319: 1,
         367: 1,
         409: 2,
         432: 1,
         520: 2,
         534: 1,
         645: 1,
         902: 1,
         942: 1,
         1278: 1,
         1414: 1,
         1642: 1,
         1674: 1,
         1794: 4,
         2023: 1,
         2127: 3,
         2136: 1,
         2273: 1,
         2572: 2,
         2964: 1,
         3121: 1,
         3299: 1,
         3506: 1,
         3527: 1,
         3719: 1,
         3761: 10,
         3777: 1,
         3795: 2,
         3853: 2,
         4158: 3,
         4256: 2,
         4467: 1,
         4829: 3,
         4950: 2,
         5107: 1,
         5304: 2,
         5447: 1,
         5555: 1,
         5662: 3,
         5948: 1,
         6037: 3,
         6101: 1,
         6547: 2,
         6751: 1,
         7229: 4,
         7822: 1,
         8232: 3,
         8432: 1,
         8532: 3,
         8728: 4,
         8837: 1,
         8974: 2,
         9101: 1,
     

In [14]:
example_axon = (Syn & "presyn=3761")
example_axon.describe()
ta3.Segment()

# Anatomically localized synapse between two Segments
-> ta3.Segmentation
synapse_id           : bigint                       # synapse index within the segmentation
---
-> ta3.Segment.proj(presyn="segment_id")
-> ta3.Segment.proj(postsyn="segment_id")
synapse_x            : bigint                       # (EM voxels)
synapse_y            : bigint                       # (EM voxels)
synapse_z            : bigint                       # (EM voxels)
syn_bbox_x1          : bigint                       # (EM voxels)  - bounding box
syn_bbox_y1          : bigint                       # (EM voxels)  - bounding box
syn_bbox_z1          : bigint                       # (EM voxels)  - bounding box
syn_bbox_x2          : bigint                       # (EM voxels)  - bounding box
syn_bbox_y2          : bigint                       # (EM voxels)  - bounding box
syn_bbox_z2          : bigint                       # (EM voxels)  - bounding box



segmentation segmentation id,segment_id segment id unique within each Segmentation,prev_segment_id segment id in previous Segmentation version
1,2,None
1,3,None
1,5,None
1,9,None
1,12,None
1,23,None
1,30,None


In [13]:
#check which ones are dendrites and which ones are axons
ta3.Neurite()
#ta3.Neurite().describe()
#(ta3.Neurite & "segment_id=4302")

segmentation segmentation id,segment_id segment id unique within each Segmentation,neurite_type
1,31,axon
1,71,axon
1,85,dendrite
1,108,axon
1,184,axon
1,269,axon
1,319,axon


In [10]:
#ta3.Soma().describe();
ta3.Soma() & "segment_id=4302"

# A segment including a cell soma
-> ta3.Segment
---
-> nda.EMCell
soma_x=null          : int                          # x location of soma in 3.58nm grid coordinates
soma_y=null          : int                          # y location of soma in 3.58nm grid coordinates
soma_z=null          : int                          # z location of soma in 3.58nm grid coordinates



segmentation segmentation id,segment_id segment id unique within each Segmentation,em_id mask ID,soma_x x location of soma in 3.58nm grid coordinates,soma_y y location of soma in 3.58nm grid coordinates,soma_z z location of soma in 3.58nm grid coordinates


In [9]:
schema = dj.schema('microns_nda',create_tables=False)
nda= dj.create_virtual_module('nda', 'microns_nda')
nda.EMCell().describe()

# cell ids in electron microscopy volume
em_id                : smallint                     # mask ID



'# cell ids in electron microscopy volume\nem_id                : smallint                     # mask ID\n'

In [16]:
nda.Trace.describe();

# raw traces extracted from cell masks
-> nda.Mask
---
trace                : longblob                     # raw calcium trace (int16 vector)



In [5]:
(nda.Trace & "scan_idx=3")

scan_idx scan ID,segmentation segmentation id,segment_id segment id unique within each Segmentation,trace raw calcium trace (int16 vector),trace_denoised
3,1,694582,=BLOB=,=BLOB=
3,1,942272,=BLOB=,=BLOB=
3,1,961113,=BLOB=,=BLOB=
3,1,2589588,=BLOB=,=BLOB=
3,1,2720227,=BLOB=,=BLOB=
3,1,3472965,=BLOB=,=BLOB=
3,1,3647513,=BLOB=,=BLOB=


In [6]:
nda.VonMises()

scan_idx scan ID,segmentation segmentation id,segment_id segment id unique within each Segmentation,von_r2 r-squared explaned by vonMises fit,von_pref preferred directions,von_base von mises base value,von_amp1 amplitude of first peak,von_amp2 amplitude of second peak,von_sharp sharpnesses,von_pvalue p-value by shuffling (nShuffles = 1e4)
2,1,694582,0.0390474,0.0879617,0.00519619,0.0612943,0.0331063,46.3828,0.0214895
2,1,961113,0.0247101,3.63509,2.88523e-05,0.0328841,0.0227009,7.17096,0.142429
2,1,1519148,0.0358466,1.23557,0.00113802,0.0391052,0.00115531,9.69029,0.0554725
2,1,2450307,0.0271941,2.4022,0.00410094,0.0374917,0.0230603,46.3828,0.143429
2,1,2589588,0.053372,3.02107,0.00165716,0.0787312,0.0374586,26.5923,0.006497
2,1,3813916,0.0578909,2.90859,0.017539,0.139912,0.0,16.2428,0.0014995
2,1,5386310,0.0710836,2.6053,0.000644453,0.215864,0.108693,10.2717,0.002499
